## Siamese

In [1]:
import os
os.chdir('../../')

In [2]:
from src.siamese import *
import torch.utils.data as data
from PIL import Image, ImageOps
import pickle
import numpy as np
from torchvision import transforms
import json
os.environ["CUDA_VISIBLE_DEVICES"]="1"

- Create croppped logo for sampled 1000 phishing

In [3]:
# data_folder = '/home/l/liny/ruofan/phishpedia/benchmark/Sampled_phish1000/'
# annot_file = '/home/l/liny/ruofan/phishpedia/benchmark/phish1000_coord.txt'
# os.makedirs('/home/l/liny/ruofan/phishpedia/benchmark/Sample_phish1000_crop', exist_ok=True)

# for brand in os.listdir(data_folder):
#     img = Image.open(data_folder + brand + '/shot.png')

#     ## get ground-truth 
#     with open(annot_file, 'r') as annotation_file:
#         for num, line in enumerate(annotation_file):
#             annotation = line.strip().split(',')
#             site = ','.join(annotation[:-4])
#             if site == brand:
#                 bbox_data_gt = np.array(list(annotation[-4:]))
#                 if len(bbox_data_gt) != 0:
#                     bboxes_gt = bbox_data_gt[:4]
#                     x_min_gt, y_min_gt, x_max_gt, y_max_gt = list(map(float, bboxes_gt))
#                     gt_bbox = [x_min_gt, y_min_gt, x_max_gt, y_max_gt]
#                     break   
# #     print(gt_bbox)
#     cropped = img.crop((x_min_gt, y_min_gt, x_max_gt, y_max_gt))
#     cropped.save(os.path.join('/home/l/liny/ruofan/phishpedia/benchmark/Sample_phish1000_crop', brand+'.png'))
#     del gt_bbox


In [4]:
# data_folder = '/home/l/liny/ruofan/phishpedia/benchmark/Sample_benign1000/'
# annot_file = '/home/l/liny/ruofan/phishpedia/benchmark/benign1000_coord.txt'
# os.makedirs('/home/l/liny/ruofan/phishpedia/benchmark/Sample_benign1000_crop', exist_ok=True)

# for brand in os.listdir(data_folder):
#     img = Image.open(data_folder + brand + '/shot.png')

#     ## get ground-truth 
#     with open(annot_file, 'r') as annotation_file:
#         for num, line in enumerate(annotation_file):
#             annotation = line.strip().split(',')
#             site = ','.join(annotation[:-4])
#             if site == brand:
#                 bbox_data_gt = np.array(list(annotation[-4:]))
#                 if len(bbox_data_gt) != 0:
#                     bboxes_gt = bbox_data_gt[:4]
#                     x_min_gt, y_min_gt, x_max_gt, y_max_gt = list(map(float, bboxes_gt))
#                     gt_bbox = [x_min_gt, y_min_gt, x_max_gt, y_max_gt]
#                     break   
# #     print(gt_bbox)
#     cropped = img.crop((x_min_gt, y_min_gt, x_max_gt, y_max_gt))
#     cropped.save(os.path.join('/home/l/liny/ruofan/phishpedia/benchmark/Sample_benign1000_crop', brand+'.png'))
#     del gt_bbox



- Define dataloader

In [3]:
class GetLoader(data.Dataset):
    def __init__(self, data_root, label_dict, transform=None, grayscale=False):
        
        self.transform = transform
        self.data_root = data_root
        self.grayscale = grayscale

        with open(label_dict, 'rb') as handle:
            self.label_dict = pickle.load(handle)

        self.classes = list(self.label_dict.keys())

        self.n_data = len(os.listdir(self.data_root))

        self.img_paths = []
        self.labels = []

        for data in os.listdir(self.data_root):
            image_path = data
            label = image_path.split('+')[0]
            
            if brand_converter(label) == 'Microsoft':
                self.labels.append(label)
                
            elif brand_converter(label) == 'DHL Airways':
                self.labels.append('DHL')
                
            elif brand_converter(label) == 'DGI French Tax Authority':
                self.labels.append('DGI (French Tax Authority)')
                
            else:
                self.labels.append(brand_converter(label))

            self.img_paths.append(image_path)

    def __getitem__(self, item):

        img_path, label= self.img_paths[item], self.labels[item]
        img_path_full = os.path.join(self.data_root, img_path)
        
        if self.grayscale:
            img = Image.open(img_path_full).convert('L').convert('RGB')
        else:
            img = Image.open(img_path_full).convert('RGB')

        img = ImageOps.expand(img, (
            (max(img.size) - img.size[0]) // 2, (max(img.size) - img.size[1]) // 2,
            (max(img.size) - img.size[0]) // 2, (max(img.size) - img.size[1]) // 2), fill=(255, 255, 255))

        # label = np.array(label,dtype='float32')
        label = self.label_dict[label]
        if self.transform is not None:
            img = self.transform(img)

        return img, label

    def __len__(self):
        return self.n_data

- Load data

In [4]:
mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]

img_transforms = transforms.Compose(
    [transforms.Resize((95, 95)),
     transforms.ToTensor(),
     transforms.Normalize(mean=mean, std=std),
    ])

valid_set = GetLoader(data_root='/home/l/liny/ruofan/phishpedia/benchmark/Sample_phish1000_crop/', 
                    label_dict='/home/l/liny/ruofan/PhishIntention/src/siamese_retrain/target_dict.json',
                     transform=img_transforms)


In [5]:
with open('/home/l/liny/ruofan/PhishIntention/src/siamese_retrain/target_dict.json', 'rb') as handle:
    label_dict = pickle.load(handle)

In [6]:
# valid_loader = torch.utils.data.DataLoader(
#   valid_set, batch_size=128, shuffle=False, pin_memory=True, drop_last=False)

valid_loader = torch.utils.data.DataLoader(
  valid_set, batch_size=1, shuffle=False, pin_memory=True, drop_last=False)

In [7]:
len(valid_loader)

1000

- Accuracy function

In [9]:
def compute_acc(dataloader, model, device):
    correct = 0
    total = 0

    for b, (x, y) in tqdm(enumerate(dataloader)):
        with torch.no_grad():
            x = x.to(device, non_blocking=True)
            y = y.to(device, non_blocking=True)
            logits = model(x)
            pred_cls = torch.argmax(logits, dim=1)

            correct += torch.sum(torch.eq(pred_cls, y)).item()
            total += y.shape[0]
            
    print('Accuracy after changing relu function: {:.2f}'.format(correct/total))    
    return correct/total

- Load model (original)

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Initialize model
model = KNOWN_MODELS["BiT-M-R50x1"](head_size=277, zero_head=True)

# Load weights
checkpoint = torch.load('/home/l/liny/ruofan/PhishIntention/src/phishpedia/resnetv2_rgb_new.pth.tar', 
                        map_location="cpu")["model"]

from collections import OrderedDict
new_state_dict = OrderedDict()
for k, v in checkpoint.items():
    name = k[7:] # remove `module.`
    new_state_dict[name] = v

model.load_state_dict(new_state_dict)

model.to(device)
model.eval()

ResNetV2(
  (root): Sequential(
    (conv): StdConv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (pad): ConstantPad2d(padding=(1, 1, 1, 1), value=0)
    (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (body): Sequential(
    (block1): Sequential(
      (unit01): PreActBottleneck(
        (gn1): GroupNorm(32, 64, eps=1e-05, affine=True)
        (conv1): StdConv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (gn2): GroupNorm(32, 64, eps=1e-05, affine=True)
        (conv2): StdConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (gn3): GroupNorm(32, 64, eps=1e-05, affine=True)
        (conv3): StdConv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (relu): ReLU(inplace=True)
        (downsample): StdConv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (unit02): PreActBottleneck(
        (gn1): GroupNorm(32, 256, eps=1e-05, aff

In [102]:
compute_acc(valid_loader, model, device)

1000it [01:38, 10.20it/s]

Accuracy after changing relu function: 0.91


0.907

## Load model (change activation function)

In [8]:
import torch
import torch.nn.functional as F
from torch import nn

In [9]:
class QuantizeRelu(nn.Module):
    def __init__(self, step_size = 0.01):
        super().__init__()
        self.step_size = step_size

    def forward(self, x):
        mask = torch.ge(x, 0).bool() # mask for positive values
        quantize = torch.ones_like(x) * self.step_size
        out = torch.mul(torch.floor(torch.div(x, quantize)), self.step_size) # quantize by step_size
        out = torch.mul(out, mask) # zero-out negative values
        out = torch.abs(out) # remove sign
        return out

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Initialize model
model = KNOWN_MODELS["BiT-M-R50x1"](head_size=180, zero_head=True)
# Load weights
model.load_state_dict(torch.load('/home/l/liny/ruofan/phishpedia/siamese/checkpoints/resnetv2_rgb.pth', map_location=device))

# replace relu with defenselayer 
# model.body.block1.unit01.relu = QuantizeRelu()
# model.body.block1.unit02.relu = QuantizeRelu()
# model.body.block1.unit03.relu = QuantizeRelu()

# model.body.block2.unit01.relu = QuantizeRelu()
# model.body.block2.unit02.relu = QuantizeRelu()
# model.body.block2.unit03.relu = QuantizeRelu()
# model.body.block2.unit04.relu = QuantizeRelu()

# model.body.block3.unit01.relu = QuantizeRelu()
# model.body.block3.unit02.relu = QuantizeRelu()
# model.body.block3.unit03.relu = QuantizeRelu()
# model.body.block3.unit04.relu = QuantizeRelu()
# model.body.block3.unit05.relu = QuantizeRelu()
# model.body.block3.unit06.relu = QuantizeRelu()

model.body.block4.unit01.relu = QuantizeRelu()
model.body.block4.unit02.relu = QuantizeRelu()
model.body.block4.unit03.relu = QuantizeRelu()

model.to(device)
model.eval()

ResNetV2(
  (root): Sequential(
    (conv): StdConv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (pad): ConstantPad2d(padding=(1, 1, 1, 1), value=0)
    (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (body): Sequential(
    (block1): Sequential(
      (unit01): PreActBottleneck(
        (gn1): GroupNorm(32, 64, eps=1e-05, affine=True)
        (conv1): StdConv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (gn2): GroupNorm(32, 64, eps=1e-05, affine=True)
        (conv2): StdConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (gn3): GroupNorm(32, 64, eps=1e-05, affine=True)
        (conv3): StdConv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (relu): ReLU(inplace=True)
        (downsample): StdConv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (unit02): PreActBottleneck(
        (gn1): GroupNorm(32, 256, eps=1e-05, aff

In [19]:
# compute_acc(valid_loader, model, device)

Accuracy after changing relu function: 0.93


0.9345417925478349

## Attack

In [10]:
from src.adv_attack.attack.Attack import *

criterion = nn.CrossEntropyLoss()
check = adversarial_attack(method='jsma', model=model, dataloader=valid_loader, 
                           device=device, num_classes=277, save_data=True)

acc, _ = check.batch_attack()

0it [00:00, ?it/s]

tensor([40], device='cuda:0')


1it [14:59, 899.33s/it]

0
Test Accuracy = 1.0
tensor([228], device='cuda:0')


1it [22:28, 1348.49s/it]


KeyboardInterrupt: 

In [16]:
acc

0.0

## BPDA

- With defense

In [10]:
from advertorch.bpda import BPDAWrapper
from advertorch.attacks import LinfPGDAttack

In [25]:
# original model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = KNOWN_MODELS["BiT-M-R50x1"](head_size=180, zero_head=True)
model.load_state_dict(torch.load('/home/l/liny/ruofan/phishpedia/siamese/checkpoints/resnetv2_rgb.pth', map_location=device))
model.to(device)

# BPDA approximation of gradients
defense_layer = BPDAWrapper(QuantizeRelu())

# replace relu with defenselayer 
model.body.block1.unit01.relu = defense_layer
model.body.block1.unit02.relu = defense_layer
model.body.block1.unit03.relu = defense_layer

model.body.block2.unit01.relu = defense_layer
model.body.block2.unit02.relu = defense_layer
model.body.block2.unit03.relu = defense_layer
model.body.block2.unit04.relu = defense_layer

model.body.block3.unit01.relu = defense_layer
model.body.block3.unit02.relu = defense_layer
model.body.block3.unit03.relu = defense_layer
model.body.block3.unit04.relu = defense_layer
model.body.block3.unit05.relu = defense_layer
model.body.block3.unit06.relu = defense_layer

model.body.block4.unit01.relu = defense_layer
model.body.block4.unit02.relu = defense_layer
model.body.block4.unit03.relu = defense_layer

bpda_adversary = LinfPGDAttack(
    model, loss_fn=nn.CrossEntropyLoss(reduction="sum"), eps=0.05,
    nb_iter=100, eps_iter=0.005, rand_init=True, clip_min=0.0, clip_max=1.0,
    targeted=False)

In [27]:
perturb_correct = 0
total = 0

for cln_data, true_label in tqdm(valid_loader):
    cln_data, true_label = cln_data.to(device), true_label.to(device)
    bpda_adv = bpda_adversary.perturb(cln_data, true_label)
    
    logits = model(bpda_adv)
    pred_cls = torch.argmax(logits, dim=1)
    perturb_correct += torch.sum(torch.eq(pred_cls, true_label)).item()
    total += len(true_label)
    
    print(perturb_correct/total)

 12%|█▎        | 1/8 [21:33<2:30:57, 1293.95s/it]

0.890625


 25%|██▌       | 2/8 [44:51<2:12:30, 1325.09s/it]

0.9140625


 38%|███▊      | 3/8 [1:08:51<1:53:17, 1359.44s/it]

...

0.921875


 88%|████████▊ | 7/8 [2:43:00<23:17, 1397.87s/it]

0.9274553571428571


100%|██████████| 8/8 [2:58:41<00:00, 1340.19s/it]

0.9274924471299094


In [28]:
perturb_correct/total

0.9274924471299094

In [24]:
perturb_correct/total

0.905337361530715

In [14]:
torch.randint(0, 277, (1,))

tensor([203])